In [ ]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2
# for visualization self-attention
!pip install -q bertviz~=1.4.0

## 編碼器 (The Encoder)

以模型 `bert-base-uncased` 為例，從分詞後的文本中獲取每個標記 (Token) 的嵌入 (Embedding)。對每個標記，生成一個大小為 $\ d_{model} = 768 $ 的向量。經過編碼器後會將意思接近的詞，產生相近的嵌入向量，且其大小與輸入相同。

In [51]:
# 載入 BERT tokenizer
from transformers import AutoTokenizer
from pprint import pprint

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

### 初探編碼器

透過一個小實驗來驗證這個結論，給定一句範例文本，包含多語意的文字 `bank`：

In [52]:
# 給定一句範例文本，包含多語義的文字 'bank'
text = '''
After stealing money from the bank vault, the bank robber was seen
fishing on the Mississippi river bank.
'''

這個詞有兩種不同的意思，一個是指銀行，另一個是指河岸。我們可以使用 BERT 模型來獲取這個詞的嵌入向量，並計算這兩個詞的餘弦相似度。如果 BERT 編碼器能夠將這兩個詞的嵌入向量分開，那麼這兩個詞的餘弦相似度 (cosine similarity) 應該偏低。

In [53]:
# 將原始文字轉換成標記
tokenized_inputs = tokenizer(
    text,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

將標記轉換回原始文本，我們可以看到 `bank` 這個詞在兩個不同的上下文中卻有相同的標記。請注意，這裡的 Token ID 是對應到 vocab 的 index，此時 `bank` 這個字具有相同的 Token ID。

In [ ]:
# Display the words with their indeces.
for id in tokenized_inputs["input_ids"][0]:
    pprint('{:<12} {:>6,}'.format(tokenizer.decode(id), id))

此刻，我們已經將原始文本轉換為標記，總計有 22 個標記。這些標記將被送入 Embedding 層得到對應的嵌入 (Embedding)。對每個標記，生成一個大小為 $\ d_{model} = 768 $ 的向量作為編碼器的輸入。

In [ ]:
len(tokenized_inputs["input_ids"][0])

#### 詞嵌入 (Word Embedding)

以下程式碼使用 BERT 模型處理文本，並獲取 `last_hidden_states` 隱藏狀態，這是一個包含最終標記嵌入的張量 (tensor)，可以幫助我們了解模型編碼器的工作原理。

為了方便我們深入觀察模型，我們透過 `output_hidden_states=True` 參數來獲取模型的隱藏狀態（hidden states），隱藏狀態是模型在每一層計算出的中間表示。

使用 `torch.no_grad()` 禁用梯度計算，這樣可以節省內存並加快計算速度，因為我們只需要前向傳播的結果，通常在推論 (Inference) 時使用。

In [ ]:
from transformers import AutoModel

# 載入 BERT model
model = AutoModel.from_pretrained(
  model_name,
  output_hidden_states=True,  # Whether the model returns all hidden-states.
)

# 這個 model 有 12 層的 transformer blocks, 及第一層的 embedding 層, 共 13 層
model

In [57]:
import torch

# Run the text through BERT, and collect 'last_hidden_state' produced from all layers.
with torch.no_grad():
    outputs = model(**tokenized_inputs)
    hidden_state = outputs.hidden_states
    # When passing output_hidden_states=True
    # you may expect the outputs.hidden_states[-1] to match outputs.last_hidden_state exactly.
    # However, this is not always the case.
    # Some models apply normalization or subsequent process to the last hidden state when it’s returned.
    last_hidden_state = outputs.last_hidden_state

# 我們只關注第一層和最終層
first_hidden_state = hidden_state[0]

根據[文件](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel)，隱藏狀態的維度是 `(batch_size, sequence_length, hidden_size)`，其中 `hidden_size` 是模型的維度，這裡是 768。

In [ ]:
last_hidden_state.shape

這個模型的完整隱藏狀態集，存儲在 `hidden_states` 物件中，這個物件有四個維度，順序如下：

1. 層數（13 層），第一層是嵌入，其餘 12 層是編碼層。
2. 批次數（1 句話）
3. 詞/標記 (Token) 數（我們句子中的 22 個 Token）
4. 隱藏單元/特徵數（768 個特徵）

因為我們不需要 `batch_size`，所以移除 `first_hidden_state` 及 `last_hidden_state` 張量中指定維度 `dim=0` 的單維度。相當於移除第一個維度的 `batch_size`。

In [ ]:
# Let’s get rid of the “batches” dimension since we don’t need it.
# Remove dimension 1, the "batches".
first_hidden_state = torch.squeeze(first_hidden_state, dim=0)
last_hidden_state = torch.squeeze(last_hidden_state, dim=0)

# 新的張量的尺寸（shape）
last_hidden_state.shape

為了確認這些向量的值確實是依賴於上下文的，讓我們看看在示例中「bank」這個詞的不同實例：

「After stealing money from the *bank vault*, the *bank robber* was seen fishing on the Mississippi *river bank*.」

讓我們找出這個示例中三個「bank」實例的位置索引。

In [ ]:
for i, id in enumerate(tokenized_inputs["input_ids"][0]):
    pprint(f'{i} {tokenizer.decode(id)}')

分別坐落在第 6, 10, 19 個標記位置。

第一層的隱藏狀態是一個 22x768 的張量，我們可以從中提取這三個「bank」的嵌入向量。

In [ ]:
# We can try printing out their vectors to compare them.
# First 6 vector values for each instance of "bank".
pprint(f"bank vault:  {first_hidden_state[6][:6]}")
pprint(f"bank robber: {first_hidden_state[10][:6]}")
pprint(f"river bank:  {first_hidden_state[19][:6]}")

同樣可以提取最終層的隱藏狀態。

In [ ]:
# We can try printing out their vectors to compare them.
# First 6 vector values for each instance of "bank".
pprint(f"bank vault:  {last_hidden_state[6][:6]}")
pprint(f"bank robber: {last_hidden_state[10][:6]}")
pprint(f"river bank:  {last_hidden_state[19][:6]}")


#### 餘弦相似度 (Cosine Similarity)

我們可以看到這些值有所不同，但讓我們計算向量之間的餘弦相似度來進行更精確的比較。

餘弦相似度測量高維空間中兩個向量之間的角度的餘弦值。它的取值範圍在 -1 和 1 之間，1 表示兩個向量是相同的，0 表示它們是正交的（即，它們之間沒有相關性）。

![](https://kdb.ai/files/2024/01/similarity-1536x696.png)

我們先看第一層的隱藏狀態，然後再看最終層的隱藏狀態。

In [ ]:
from scipy.spatial.distance import cosine

# 比較第一層 'bank robber' 與 'bank vault' 的相似度
cosine_similarity = 1 - cosine(first_hidden_state[6], first_hidden_state[10])
pprint(f"Similarity between 'bank robber' and 'bank vault' in the first layer: {cosine_similarity:.4f}")

In [ ]:
# 比較第一層 'bank robber' 與 'river bank' 的相似度
cosine_similarity = 1 - cosine(first_hidden_state[10], first_hidden_state[19])
pprint(f"Similarity between 'bank robber' and 'river bank' in the first layer: {cosine_similarity:.4f}")

從第一層 Embedding 層中提取的「bank」的嵌入向量，我們可以看到這兩個向量的餘弦相似度是相當接近的。

我們再來看經過編碼器處理後的最終層的隱藏狀態。

In [ ]:
# 比較 'bank robber' 與 'bank vault' 的相似度
same_bank = 1 - cosine(last_hidden_state[10], last_hidden_state[6])
pprint('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)

In [ ]:
# 比較 'bank robber' 與 'river bank' 的相似度
diff_bank = 1 - cosine(last_hidden_state[10], last_hidden_state[19])
pprint('Vector similarity for *different* meanings:  %.2f' % diff_bank)

透過這個小實驗我們觀察到

1. 分詞器 (Tokenizer) 會將相同的詞轉換為相同的標記 (Token)，這是因為字典是固定的。
2. Embedding 層會為每個標記生成一個向量作為編碼器的輸入，但除了位置差異外，這些向量在不同的上下文中是相似的。
3. 編碼器針對詞意的相近程度產生一個新的向量，將不同詞意的詞區分開來。

初步了解編碼器的作用後，接下來我們將深入探討編碼器的結構。

Transformer 由 N 個編碼器 (Encoder) 組成，每個編碼器將其輸出送到下一個編碼器。最終的編碼器返回 **輸入** 的 **向量** 表示。為了說明，從現在開始，我們將使用 N=2 的值。

![encoder in a nutshell](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoders.png)

每個編碼器區塊由兩個子層組成：

1. 多頭注意力 (Multi-head Attention)
2. 前饋神經網路 (Feedforward Network)

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoders.inside-768x315.png)

在開始解釋這兩個組件之前，有必要先了解自注意力 (Self-Attention) 機制。

### 自注意力 (Self-Attention)

考慮以下句子：

In [67]:
text = "John and Paul wrote several songs when they were inspired."

在這個句子中，自注意力機制計算每個詞的表示，並且與句子中其他詞的關係提供了更多關於該詞的信息。例如，「they」這個詞應該與「John」和「Paul」相關，而不是與「songs」相關。

讓我們簡單以一個視覺化的方式來解釋自注意力機制：

首先，我們透過熟悉的手法將文本經過分詞器獲得標記 (Token)，只是這次我們使用 `output_attentions=True` 在前向傳播時會返回注意力權重（attention weights）。注意力權重是模型在計算注意力機制時生成的權重矩陣，這些權重表示模型在計算每個輸出時對輸入的不同部分的關注程度。這樣可以在需要時訪問模型的注意力機制，便於進行更深入的分析和調試。注意力權重可以幫助我們理解模型在處理輸入數據時的關注點和決策過程。

In [68]:
# 對輸入文本進行 tokenization
tokenized_inputs = tokenizer(
    text,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

model = AutoModel.from_pretrained(
    model_name,
    output_attentions=True, # 取得注意力權重
)

以下程式碼使用 BERT 模型處理文本，並收集注意力權重。

使用 `torch.no_grad()` 禁用梯度計算，這樣可以節省內存並加快計算速度，因為我們只需要前向傳播的結果。

In [69]:
# Run the text through BERT, and collect all of attention weights
with torch.no_grad():
    outputs = model(**tokenized_inputs)
    # 取得注意力權重
    attention = outputs.attentions

In [ ]:
n_layers = len(attention)
pprint(f"Number of attention layers: {n_layers}")

根據[文件](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel)，每一層的注意力權重的維度是 `(batch_size, num_heads, sequence_length, sequence_length)`。

In [ ]:
attention[0].size()

#### 視覺化注意力權重

In [ ]:
# 將 token id 轉換回 token
tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs['input_ids'][0])
pprint(tokens, compact=True)

In [ ]:
from bertviz import (
  model_view,
  head_view,
)

# 圖示化模型第 8 層的第 9 個 head 的自注意力機制
# Review the relationship of term 'they' along with 'John' and 'Paul'
head_view(
  attention,
  tokens,
  layer=8,
  heads=[9],
)

以一個更簡單的例子「The sky is blue」來理解自注意力機制如何運作，編碼器 (Encoder) 接收句子中每個詞的維度為 $\ d_{model} = 512 $ 的 Word Embedding 向量，例如：

$\ x_1 = [3.23, 0.65, ..., 4.78] \Rightarrow \text{"The"} $

$\ x_2 = [1.26, 6.35, ..., 7.99] \Rightarrow \text{"sky"} $

$\ x_3 = [9.25, 1.68, ..., 4.26] \Rightarrow \text{"is"} $

$\ x_4 = [6.84, 2.98, ..., 11.48] \Rightarrow \text{"blue"} $

有了這些向量，我們可以組裝 Embedding 矩陣 $\ X $，其維度為$\ d = [4 \times 512] $：

![](https://alexisalulema.com/wp-content/uploads/2022/08/embedding_matrix.gif)

我們將從這個矩陣 $\ X $ 中創建三個額外的矩陣，作為「自注意力機制」的一部分：

* $\ Q $, query matrix
* $\ K $, key matrix
* $\ V $, value matrix

要創建這些陣列，我們還需要三個新的權重矩陣 (weight matrices)。

原始論文中使用的維度是 $\ d_k = 64 $；因此，權重向量的維度將是 $\ d_{model} \times d_k \Rightarrow 512 \times 64 $，這些矩陣會用隨機值初始化：

* $\ W^Q $, query weight matrix
* $\ W^K $, key weight matrix
* $\ W^V $, value weight matrix

權重矩陣攜帶了在訓練過程中學到的最佳值，因此每個矩陣 $\ Q $、$\ K $ 和 $\ V $ 是 Embedding 矩陣 $\ X $ 與相應權重矩陣的乘積，這會生成 $\ 4 \times 64 $ 的矩陣：

* $\ Q = X × W^Q $
* $\ K = X × W^K $
* $\ V = X × W^V $ 

![](https://alexisalulema.com/wp-content/uploads/2022/08/QKV.gif)

每個矩陣中的四行分別代表句子「The sky is blue」中的每個詞。

#### Self-attention Mechanism Process

1. 計算點積 (dot product) $\ Q \cdot K^T $

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/QdotKT-1-1024x140.gif)

結果陣列的元素表示詞語之間的關係。例如，$\ q_1 \cdot k_1 $ 是詞「The」與其自身的關係，$\ q_1 \cdot k_3 $ 是「The」與「is」之間的關係。「sky」與「blue」之間的關係 $\ q_2 \cdot k_4 $ 會有稍高的值，因為名詞和形容詞之間存在關係。例如：

![](https://alexisalulema.com/wp-content/uploads/2022/08/QdotKT.2.gif)

這樣，我們可以說計算查詢矩陣 $\ Q $ 和鍵矩陣 $\ K^T $ 之間的點積，基本上給出了相似度值，這有助於我們理解句子中每個詞與所有其他詞的相似程度。

2. 計算 $\ QK^T / \sqrt{d_k} $。這個操作有助於獲得穩定的梯度，其中 $\ d_k = 64 $ 是鍵向量的維度。

![](https://alexisalulema.com/wp-content/uploads/2022/08/QdotKTSqrtDK.gif)

結果矩陣的值必須正規化，如果我們使用函數 $\ \text{Softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) $，我們可以將值轉化為 0 到 1 的範圍。

每行的值之和等於 1。通過這些值，我們可以理解句子中每個詞與所有其他詞的關係。這稱為分數矩陣 (score matrix)。

3. 接下來，我們需要計算注意力矩陣 (attention matrix) $\ Z $：

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/SoftmaxV.gif)

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/Z.SoftmaxV-1024x148.gif)

注意力矩陣 $\ Z $ 是一個具有 4 行和 512 列的矩陣，對於示例句子來說。每行對應於相應詞的自注意力 (self-attention) 向量。

自注意力 (self-attention) 機制被稱為縮放點積注意力 (scaled dot product attention)，因為我們在計算向量 $\ Q $ 和 $\ K $ 之間的點積並通過 $\ \sqrt{d_k} $ 來縮放值。

我們也可以透過視覺化的方式來理解自注意力機制中 $\ Ｑ $ 和 $\ Ｋ $ 的作用：


In [ ]:
from bertviz.transformers_neuron_view import BertModel, BertTokenizer
from bertviz.neuron_view import show

tokenizer = BertTokenizer.from_pretrained(
  'bert-base-uncased',
  do_lower_case=True)
model = BertModel.from_pretrained(
  'bert-base-uncased',
  output_attentions=True)
# 圖示化第 8 層的第 9 個 head 的自注意力機制中 Q 和 K 的作用
show(
  model=model,
  model_type='bert',
  tokenizer=tokenizer,
  sentence_a=text,
  layer=8,
  head=9,
  )

### 多頭注意力 (Multi-head Attention)

對於 Transformer，我們將計算多個注意力 (multiple attention) 矩陣。但為什麼我們需要多個陣列呢？這有助於在語境中詞義模糊的情況下，例如：

```
Tom was crying because he was blue.
```

單一的注意力機制可能會決定 Tom 哭泣是因為他的顏色是藍色，這是由於「Tom」這個詞的影響。如果大多數句子中「blue」表示顏色，只有一個「注意力頭 (attention head)」的機制將正確地學習到它是一種顏色。然而，擁有「多個注意力頭 (multiple attention heads)」，其中一個注意力機制更有可能從句子中學習到「blue」表示心情，通過串聯「多個注意力頭 (multiple attention heads)」的結果，注意力矩陣將更加準確。

我們如何計算多個注意力矩陣？假設我們要計算兩個注意力矩陣：$\ Z_1 $ 和 $\ Z_2 $。

要計算 $\ Z_1 $，首先，我們創建三個矩陣 $\ Q_1 $、$\ K_1 $ 和 $\ V_1 $，這意味著將 Embedding 矩陣與三個權重矩陣 $\ W_1^Q $、$\ W_1^K $ 和 $\ W_1^V $ 相乘。現在，注意力矩陣的計算方式如下：

$\ Z_1 = \text{Softmax}\left(\frac{Q_1K_1^T}{\sqrt{d_k}} \cdot V_1 \right) $

同樣地，對於 $\ Z_2 $ 也是如此。

$\ Z_2 = \text{Softmax}\left(\frac{Q_2K_2^T}{\sqrt{d_k}} \cdot V_2 \right) $

這樣，我們可以計算任意數量的注意力矩陣。假設我們需要八個注意力矩陣，在《[Attention is all you need](https://arxiv.org/abs/1706.03762?context=cs)》中提到的數值。在這種情況下，我們可以將所有的注意力頭拼接 (concatenate) 起來，並將結果乘以一個新的權重矩陣 $\ W_0 $，該矩陣經過訓練以表示注意力機制的最佳值。

Multi-head attention $\ = Concatenate( Z_1, Z_2, Z_3, Z_4, Z_5, Z_6, Z_7, Z_8 ) \cdot W_0 $

### 前饋神經網路 (Feedforward Network)

Feedforward Network 由兩個帶有 ReLU 激活的全連接層組成。

另一個用於連接輸入和編碼器 (Encoder) 的組件是 **Add and norm** 組件，這是一種連接，隨後進行 **layer normalization**。

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/addNnorm.png)

通過 Layer normalization 防止每層中的值發生劇烈變化來實現更快的訓練。

